In [3]:
import pandas as pd
df = pd.read_csv('C:/Users/user/Desktop/보이스피싱/woogawooga_project/dataset/normal_비율맞춘_학습데이터셋.csv')
df.rename(columns={'file_name': 'file_id'}, inplace=True)

In [4]:
df.to_csv("normal_비율맞춘_학습데이터셋2.csv", index=False, encoding="utf-8-sig")


In [6]:
import os
import json
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from kiwipiepy import Kiwi

# 1. 경로 설정 및 저장 함수
save_dir = "phishing_keyword_pipeline2"
os.makedirs(save_dir, exist_ok=True)

def save_csv(df, name):
    path = os.path.join(save_dir, name)
    df.to_csv(path, index=False, encoding="utf-8-sig")
    print(f"CSV 저장: {path}")

def save_json(obj, name):
    path = os.path.join(save_dir, name)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)
    print(f"JSON 저장: {path}")

# 2. 데이터 불러오기
df_phish = pd.read_csv("C:/Users/user/Desktop/보이스피싱/woogawooga_project/dataset_create/Chaeyeon/phishing_total_0716_1345.csv")
df_normal = pd.read_csv("C:/Users/user/Desktop/보이스피싱/woogawooga_project/dataset_create/Chaeyeon/normal_비율맞춘_학습데이터셋2.csv")

# 3. 대화 단위로 묶기
phish_dialogue = df_phish.groupby("file_id").agg({
    "text": lambda x: " ".join(x),
    "phishing_type": "first"
}).reset_index()

normal_dialogue = df_normal.groupby("file_id")["text"].apply(lambda x: " ".join(x)).reset_index()

save_csv(phish_dialogue, "phish_dialogue.csv")
save_csv(normal_dialogue, "normal_dialogue.csv")

# 4. Kiwi 형태소 분석기 + NNP 제거
kiwi = Kiwi()

def tokenize(text):
    tokens = kiwi.tokenize(str(text))
    return ' '.join([
        token.lemma for token in tokens
        if (token.tag.startswith("N") or token.tag.startswith("V") or token.tag.startswith("VA"))
        and not token.tag.startswith("NNP")  # 고유명사 제거
    ])

phish_dialogue["tokenized_text"] = phish_dialogue["text"].apply(tokenize)
normal_dialogue["tokenized_text"] = normal_dialogue["text"].apply(tokenize)

save_csv(phish_dialogue, "phish_tokenized.csv")
save_csv(normal_dialogue, "normal_tokenized.csv")

# 5. TF-IDF 기반 위험도 계산 (보이스피싱 vs 일반대화)
all_docs = list(phish_dialogue["tokenized_text"]) + list(normal_dialogue["tokenized_text"])
vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = vectorizer.fit_transform(all_docs)
words = vectorizer.get_feature_names_out()

n_phish = len(phish_dialogue)
n_normal = len(normal_dialogue)
phish_avg = tfidf_matrix[:n_phish].mean(axis=0).A1
normal_avg = tfidf_matrix[n_phish:].mean(axis=0).A1

epsilon = 1e-6
risk_dict = {word: phish_avg[i] / (normal_avg[i] + epsilon) for i, word in enumerate(words)}
save_json(risk_dict, "risk_dict2.json")

# 6. 위험도 × log(등장빈도) 기반 유형별 키워드 추출
top_keywords_log = {}

for ptype, group in phish_dialogue.groupby("phishing_type"):
    tokens = ' '.join(group["tokenized_text"]).split()
    counts = Counter(tokens)

    keyword_scores = {
        word: risk_dict.get(word, 0) * np.log1p(counts[word])
        for word in counts if word in risk_dict
    }

    top50 = sorted(keyword_scores.items(), key=lambda x: x[1], reverse=True)[:50]
    top_keywords_log[ptype] = dict(top50)

save_json(top_keywords_log, "top_keywords_log2.json")


CSV 저장: phishing_keyword_pipeline2\phish_dialogue.csv
CSV 저장: phishing_keyword_pipeline2\normal_dialogue.csv
CSV 저장: phishing_keyword_pipeline2\phish_tokenized.csv
CSV 저장: phishing_keyword_pipeline2\normal_tokenized.csv
JSON 저장: phishing_keyword_pipeline2\risk_dict2.json
JSON 저장: phishing_keyword_pipeline2\top_keywords_log2.json


In [7]:
import pandas as pd
import json
from collections import Counter

# 1. 데이터 로드
phish_df = pd.read_csv("phishing_keyword_pipeline2/phish_tokenized.csv")
normal_df = pd.read_csv("phishing_keyword_pipeline2/normal_tokenized.csv")

phish_df["label"] = 1
normal_df["label"] = 0

total_df = pd.concat([phish_df, normal_df], ignore_index=True)

# 2. 키워드 로드
with open("phishing_keyword_pipeline2/top_keywords_log2.json", "r", encoding="utf-8") as f:
    keyword_dict = json.load(f)

# 3. 전체 키워드 목록 통합
all_keywords = set()
for keyword_list in keyword_dict.values():
    all_keywords.update(keyword_list.keys())

# 정렬해서 고정된 순서로 사용
all_keywords = sorted(all_keywords)

# 4. 등장 횟수 기반 feature 생성
def keyword_count_vector(text, keyword_set):
    tokens = text.split()
    counts = Counter(tokens)
    return [counts.get(word, 0) for word in keyword_set]

# 5. X, y 생성
X = total_df["tokenized_text"].apply(lambda x: keyword_count_vector(x, all_keywords))
X_df = pd.DataFrame(X.tolist(), columns=[f"kw_{kw}" for kw in all_keywords])
X_df["label"] = total_df["label"]

# 6. 저장
X_df.to_csv("phishing_keyword_pipeline2/X_train.csv", index=False, encoding="utf-8-sig")
print("X_train.csv 저장 완료!")

X_train.csv 저장 완료!


In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
import scipy

# 1. 데이터 불러오기
df = pd.read_csv("phishing_keyword_pipeline2/X_train.csv")

# 2. 레이블 분리
y = df["label"]
X_counts = df.drop(columns=["label"])

# 3. TF-IDF 가중치 적용
transformer = TfidfTransformer()
X_tfidf = transformer.fit_transform(X_counts)

# 4. 저장 (압축 희소행렬 포맷으로 저장)
scipy.sparse.save_npz("phishing_keyword_pipeline2/X_train_tfidf.npz", X_tfidf)
y.to_csv("phishing_keyword_pipeline2/y_train.csv", index=False)

print("TF-IDF 변환 완료 및 저장됨!")
print(f"TF-IDF 행렬 shape: {X_tfidf.shape}")


TF-IDF 변환 완료 및 저장됨!
TF-IDF 행렬 shape: (12386, 142)


In [12]:
import pandas as pd
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# 데이터 불러오기
X = scipy.sparse.load_npz("phishing_keyword_pipeline2/X_train_tfidf.npz")
y = pd.read_csv("phishing_keyword_pipeline2/y_train.csv").values.ravel()

# 학습/검증 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 로지스틱 회귀 모델 학습
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_val)
print("모델 성능 평가 결과")
print(classification_report(y_val, y_pred, digits=4))

# 5. 혼동 행렬 (추가 분석용)
cm = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:\n", cm)

# 6. 모델 저장 (선택)
import joblib
joblib.dump(model, "phishing_keyword_pipeline2/boryunull.pkl")


모델 성능 평가 결과
              precision    recall  f1-score   support

           0     0.9010    0.9325    0.9165      1200
           1     0.9345    0.9038    0.9189      1278

    accuracy                         0.9177      2478
   macro avg     0.9177    0.9181    0.9177      2478
weighted avg     0.9182    0.9177    0.9177      2478

Confusion Matrix:
 [[1119   81]
 [ 123 1155]]


['phishing_keyword_pipeline2/boryunull.pkl']

In [13]:
import pandas as pd
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# 1. 데이터 불러오기
X = scipy.sparse.load_npz("phishing_keyword_pipeline2/X_train_tfidf.npz")
y = pd.read_csv("phishing_keyword_pipeline2/y_train.csv").values.ravel()

# 2. 학습/검증 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 3. 로지스틱 모델 학습
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# 4. 확률 기반 예측
y_prob = model.predict_proba(X_val)[:, 1]  # 클래스 1(피싱) 확률만 가져오기

# 5. 보류 구간 처리
def classify(prob):
    if prob < 0.3:
        return 0  # 일반
    elif prob > 0.7:
        return 1  # 피싱
    else:
        return 2  # 보류

y_pred_custom = [classify(p) for p in y_prob]

# 6. 평가 출력
results_df = pd.DataFrame({
    "true": y_val,
    "prob": y_prob,
    "pred": y_pred_custom
})

# 7. 각 클래스별 개수 확인
print(" 예측 결과 분포 (0: 일반, 1: 피싱, 2: 보류):")
print(results_df["pred"].value_counts())

# 8. 혼동행렬 (보류 제외한 평가)
print("\n 보류 제외하고 정확도 평가 (0과 1만):")
mask = results_df["pred"] != 2
print(classification_report(results_df["true"][mask], results_df["pred"][mask], digits=4))

# 9. 보류된 데이터 수
n_total = len(results_df)
n_hold = (results_df["pred"] == 2).sum()
print(f"\n 전체 {n_total}개 중 보류된 데이터: {n_hold}개 ({n_hold / n_total:.1%})")

# 저장
joblib.dump(model, "phishing_keyword_pipeline2/model_logistic_v1.pkl")
print("모델 저장 완료: model_logistic_v1.pkl")

 예측 결과 분포 (0: 일반, 1: 피싱, 2: 보류):
pred
0    1139
1    1114
2     225
Name: count, dtype: int64

 보류 제외하고 정확도 평가 (0과 1만):
              precision    recall  f1-score   support

           0     0.9254    0.9582    0.9415      1100
           1     0.9587    0.9263    0.9422      1153

    accuracy                         0.9419      2253
   macro avg     0.9420    0.9422    0.9419      2253
weighted avg     0.9424    0.9419    0.9419      2253


 전체 2478개 중 보류된 데이터: 225개 (9.1%)
모델 저장 완료: model_logistic_v1.pkl


In [14]:
import pandas as pd
import scipy
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

# 1. 파일 경로
X = scipy.sparse.load_npz("phishing_keyword_pipeline2/X_train_tfidf.npz")
y = pd.read_csv("phishing_keyword_pipeline2/y_train.csv").values.ravel()

# 2. 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 3. 로지스틱 모델 학습
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_prob = model.predict_proba(X_val)[:, 1]

# 4. 다양한 보류 구간 테스트
thresholds = [(round(i, 2), round(j, 2)) for i in np.arange(0.1, 0.5, 0.05)
                                            for j in np.arange(0.6, 0.91, 0.05) if i < j]

results = []

for low, high in thresholds:
    def classify(p):
        if p < low:
            return 0
        elif p > high:
            return 1
        else:
            return 2

    y_pred = np.array([classify(p) for p in y_prob])
    mask = y_pred != 2

    if np.sum(mask) == 0:
        continue

    prec = precision_score(y_val[mask], y_pred[mask], zero_division=0)
    rec = recall_score(y_val[mask], y_pred[mask])
    f1 = f1_score(y_val[mask], y_pred[mask])
    hold_rate = 1 - np.mean(mask)

    results.append({
        "보류_하한": low,
        "보류_상한": high,
        "precision": round(prec, 4),
        "recall": round(rec, 4),
        "f1_score": round(f1, 4),
        "보류비율(%)": round(hold_rate * 100, 1)
    })

results_df = pd.DataFrame(results).sort_values(by="f1_score", ascending=False)
results_df.to_csv("phishing_keyword_pipeline2/보류구간_성능비교.csv", index=False, encoding="utf-8-sig")
print("결과 저장 완료: 보류구간_성능비교.csv")

결과 저장 완료: 보류구간_성능비교.csv


In [19]:
import pandas as pd
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# 1. 데이터 불러오기
X = scipy.sparse.load_npz("phishing_keyword_pipeline2/X_train_tfidf.npz")
y = pd.read_csv("phishing_keyword_pipeline2/y_train.csv").values.ravel()

# 2. 학습/검증 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 3. 로지스틱 모델 학습
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# 4. 확률 기반 예측
y_prob = model.predict_proba(X_val)[:, 1]  # 클래스 1(피싱) 확률만 가져오기

# 5. 보류 구간 처리
def classify(prob):
    if prob < 0.15:
        return 0  # 일반
    elif prob > 0.85:
        return 1  # 피싱
    else:
        return 2  # 보류

y_pred_custom = [classify(p) for p in y_prob]

# 6. 평가 출력
results_df = pd.DataFrame({
    "true": y_val,
    "prob": y_prob,
    "pred": y_pred_custom
})

# 7. 각 클래스별 개수 확인
print(" 예측 결과 분포 (0: 일반, 1: 피싱, 2: 보류):")
print(results_df["pred"].value_counts())

# 8. 혼동행렬 (보류 제외한 평가)
print("\n 보류 제외하고 정확도 평가 (0과 1만):")
mask = results_df["pred"] != 2
print(classification_report(results_df["true"][mask], results_df["pred"][mask], digits=4))

# 9. 보류된 데이터 수
n_total = len(results_df)
n_hold = (results_df["pred"] == 2).sum()
print(f"\n 전체 {n_total}개 중 보류된 데이터: {n_hold}개 ({n_hold / n_total:.1%})")


 예측 결과 분포 (0: 일반, 1: 피싱, 2: 보류):
pred
0    1096
1     955
2     427
Name: count, dtype: int64

 보류 제외하고 정확도 평가 (0과 1만):
              precision    recall  f1-score   support

           0     0.9343    0.9808    0.9570      1044
           1     0.9791    0.9285    0.9531      1007

    accuracy                         0.9551      2051
   macro avg     0.9567    0.9547    0.9551      2051
weighted avg     0.9563    0.9551    0.9551      2051


 전체 2478개 중 보류된 데이터: 427개 (17.2%)


In [23]:
import pandas as pd
import numpy as np
import scipy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 1. 데이터 로드
X = scipy.sparse.load_npz("phishing_keyword_pipeline2/X_train_tfidf.npz")
y = pd.read_csv("phishing_keyword_pipeline2/y_train.csv").values.ravel()

# 2. train/test 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 3. 랜덤 포레스트 모델 학습
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# 4. 예측 확률
y_prob = rf_model.predict_proba(X_val)[:, 1]

# 5. 보류 구간 기준 예측 (예: 0.10 ~ 0.75)
def classify(p, low=0.10, high=0.9):
    if p < low:
        return 0
    elif p > high:
        return 1
    else:
        return 2

y_pred_custom = np.array([classify(p) for p in y_prob])

# 6. 평가 (보류 제외)
results_df = pd.DataFrame({
    "true": y_val,
    "prob": y_prob,
    "pred": y_pred_custom
})

mask = results_df["pred"] != 2
print("Random Forest 결과 (보류 제외)")
print(classification_report(results_df["true"][mask], results_df["pred"][mask], digits=4))
print(f"보류 비율: {(results_df['pred'] == 2).mean() * 100:.1f}%")

# 7. 모델 저장 (선택)
import joblib
joblib.dump(rf_model, "phishing_keyword_pipeline2/model_randomforest_v1.pkl")


Random Forest 결과 (보류 제외)
              precision    recall  f1-score   support

           0     0.9335    0.9924    0.9620      1046
           1     0.9923    0.9332    0.9618      1107

    accuracy                         0.9619      2153
   macro avg     0.9629    0.9628    0.9619      2153
weighted avg     0.9637    0.9619    0.9619      2153

보류 비율: 13.1%


['phishing_keyword_pipeline2/model_randomforest_v1.pkl']

In [22]:
import pandas as pd
import numpy as np
import scipy
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

# 1. 데이터 로딩
X = scipy.sparse.load_npz("phishing_keyword_pipeline2/X_train_tfidf.npz")
y = pd.read_csv("phishing_keyword_pipeline2/y_train.csv").values.ravel()

# 2. 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 3. 랜덤 포레스트 모델 학습
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# 4. 예측 확률
y_prob = rf_model.predict_proba(X_val)[:, 1]

# 5. 다양한 보류 구간 실험
thresholds = [(round(i, 2), round(j, 2)) for i in np.arange(0.1, 0.5, 0.05)
                                            for j in np.arange(0.6, 0.91, 0.05) if i < j]

results = []

for low, high in thresholds:
    def classify(p):
        if p < low:
            return 0
        elif p > high:
            return 1
        else:
            return 2

    y_pred = np.array([classify(p) for p in y_prob])
    mask = y_pred != 2

    if np.sum(mask) == 0:
        continue

    prec = precision_score(y_val[mask], y_pred[mask], zero_division=0)
    rec = recall_score(y_val[mask], y_pred[mask])
    f1 = f1_score(y_val[mask], y_pred[mask])
    hold_rate = 1 - np.mean(mask)

    results.append({
        "보류_하한": low,
        "보류_상한": high,
        "precision": round(prec, 4),
        "recall": round(rec, 4),
        "f1_score": round(f1, 4),
        "보류비율(%)": round(hold_rate * 100, 1)
    })

# 6. 저장
results_df = pd.DataFrame(results).sort_values(by="f1_score", ascending=False)
results_df.to_csv("phishing_keyword_pipeline2/RandomForest_보류구간_성능비교.csv", index=False, encoding="utf-8-sig")
print("랜덤 포레스트 보류구간 실험 완료!")
print("저장됨: RandomForest_보류구간_성능비교.csv")


랜덤 포레스트 보류구간 실험 완료!
저장됨: RandomForest_보류구간_성능비교.csv


In [24]:
import pandas as pd
import scipy
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

# 1. 데이터 로딩
X = scipy.sparse.load_npz("phishing_keyword_pipeline2/X_train_tfidf.npz")
y = pd.read_csv("phishing_keyword_pipeline2/y_train.csv").values.ravel()

# 2. train/validation 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 3. XGBoost 모델 학습
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric="logloss", n_jobs=-1, random_state=42)
xgb_model.fit(X_train, y_train)

# 4. 예측 확률
y_prob = xgb_model.predict_proba(X_val)[:, 1]

# 5. 보류 구간 분류 (예: 0.10 ~ 0.75)
def classify(prob, low=0.10, high=0.75):
    if prob < low:
        return 0
    elif prob > high:
        return 1
    else:
        return 2

y_pred = np.array([classify(p) for p in y_prob])

# 6. 보류 제외 후 성능 평가
mask = y_pred != 2
print("XGBoost 결과 (보류 제외):")
print(classification_report(y_val[mask], y_pred[mask], digits=4))
print(f"전체 개수: {len(y_pred)}개 중 보류: {np.sum(y_pred == 2)}개 ({np.mean(y_pred == 2) * 100:.1f}%)")

# 7. 모델 저장
joblib.dump(xgb_model, "phishing_keyword_pipeline2/model_xgboost_v2.pkl")
print("모델 저장 완료: model_xgboost_v2.pkl")


c:\Users\user\Desktop\보이스피싱\woogawooga_project\.venv\lib\site-packages\xgboost\training.py:183: UserWarning: [17:30:09] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost 결과 (보류 제외):
              precision    recall  f1-score   support

           0     0.9324    0.9758    0.9536      1075
           1     0.9767    0.9349    0.9553      1167

    accuracy                         0.9545      2242
   macro avg     0.9546    0.9553    0.9545      2242
weighted avg     0.9555    0.9545    0.9545      2242

전체 개수: 2478개 중 보류: 236개 (9.5%)
모델 저장 완료: model_xgboost_v2.pkl
